In [1]:
import PIL
from PIL import Image
import imagehash

import pycamhd.pycamhd as camhd
import numpy as np
import matplotlib.pyplot as plt
import xarray as xar
import urllib as url

movLink = "https://rawdata.oceanobservatories.org/files/RS03ASHS/PN03B/06-CAMHDA301/2016/07/29/CAMHDA301-20160729T000000Z.mov"
movAtom = camhd.get_moov_atom(movLink)
frameTotal = camhd.get_frame_count(movLink, movAtom) 
frames = range(0,frameTotal,200)
#print(frameTotal)                        

def diff_letters(a,b):
    if (len(a) != len(b)):
        return -1
    else:
        return sum ( a[i] != b[i] for i in range(len(a)) )

ModuleNotFoundError: No module named 'imagehash'